In [ ]:
# Copy the event camera dataset from Google Drive to current directory
!cp /content/drive/MyDrive/Event_Camera_MasterThesis/timeStack_1281281_tf.zip ./
# Unzip the dataset to current directory
!unzip ./timeStack_1281281_tf.zip -d ./

In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Create symbolic link to models directory in Google Drive
!ln -s /content/drive/MyDrive/Event_Camera_MasterThesis/models models


In [ ]:
# Initialize Weights & Biases for experiment tracking
import wandb
wandb.login()  # Login to wandb account

In [ ]:
# Import TensorFlow and print version for verification
import tensorflow as tf
print(tf.__version__)

a5e0804dade6129a32e80434a145699dd5392d03 api

In [ ]:
import os
import glob
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from wandb.integration.keras import WandbCallback
import wandb
from collections import Counter

# Custom WandbCallback to skip graph recording
class CustomWandbCallback(WandbCallback):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._step = 0

    def on_train_batch_end(self, batch, logs=None):
        if logs is None:
            logs = {}
        wandb.log(logs, step=self._step)
        self._step += 1

def normalize_event_grid(event_grid):
    # Calculate min and max values for each channel in spatial dimensions
    min_vals = tf.reduce_min(event_grid, axis=[0, 1], keepdims=True)
    max_vals = tf.reduce_max(event_grid, axis=[0, 1], keepdims=True)
    # Avoid division by zero
    normalized_grid = (event_grid - min_vals) / (max_vals - min_vals + 1e-8)
    return normalized_grid

def parse_tfrecord_function(record, filename, table, num_classes):
    """
    Parse single TFRecord and extract label from file path.
    Assumes TFRecord contains 'event_grid' and 'shape' fields,
    and class name is in second-to-last position of file path.
    """
    features = {
        'event_grid': tf.io.FixedLenFeature([], tf.string),
        'shape': tf.io.FixedLenFeature([3], tf.int64)
    }
    example = tf.io.parse_single_example(record, features)
    event_grid = tf.io.decode_raw(example['event_grid'], tf.float32)
    shape = example['shape']
    event_grid = tf.reshape(event_grid, shape)
    event_grid = tf.image.resize(event_grid, (224, 224))
    event_grid = normalize_event_grid(event_grid)

    # Extract label from file path
    parts = tf.strings.split(filename, os.sep)
    label_str = parts[-2]
    label_int = table.lookup(label_str)
    label = tf.one_hot(label_int, depth=num_classes)
    return event_grid, label

def create_dataset_tf2(data_dir, batch_size, seed=42):
    """
    Build TFRecord dataset:
    - Get all TFRecord files (excluding .ipynb_checkpoints)
    - Perform stratified sampling based on parent folder names
    - Read files using tf.data.TFRecordDataset with interleave
    - Map to parsing function
    """
    all_files = glob.glob(os.path.join(data_dir, "*/*.tfrecord"))
    all_files = [f for f in all_files if ".ipynb_checkpoints" not in f]

    # Get valid classes (folder names) and sort
    valid_classes = sorted([cls for cls in os.listdir(data_dir)
                             if os.path.isdir(os.path.join(data_dir, cls)) and cls != '.ipynb_checkpoints'])
    print("Valid classes:", valid_classes)

    # Create class lookup table
    keys = tf.constant(valid_classes)
    vals = tf.constant(range(len(valid_classes)), dtype=tf.int32)
    table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys, vals), default_value=-1)
    num_classes = len(valid_classes)

    # Get labels for stratified sampling
    labels = [os.path.basename(os.path.dirname(f)) for f in all_files]

    # Split into train and validation sets
    train_files, val_files = train_test_split(
        all_files, test_size=0.2, random_state=seed, stratify=labels)

    print("Training samples:", len(train_files))
    print("Validation samples:", len(val_files))
    print("Training class distribution:", Counter([os.path.basename(os.path.dirname(f)) for f in train_files]))
    print("Validation class distribution:", Counter([os.path.basename(os.path.dirname(f)) for f in val_files]))

    def process_file(filename):
        ds = tf.data.TFRecordDataset(filename)
        ds = ds.map(lambda record: (record, filename))
        return ds

    # Build training dataset
    train_ds = tf.data.Dataset.from_tensor_slices(train_files)
    train_ds = train_ds.interleave(lambda x: process_file(x),
                                   cycle_length=tf.data.AUTOTUNE,
                                   block_length=1)
    train_ds = train_ds.map(lambda record, filename: parse_tfrecord_function(record, filename, table, num_classes),
                            num_parallel_calls=tf.data.AUTOTUNE)
    train_ds = train_ds.shuffle(1000, seed=seed).batch(batch_size, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

    # Build validation dataset
    val_ds = tf.data.Dataset.from_tensor_slices(val_files)
    val_ds = val_ds.interleave(lambda x: process_file(x),
                               cycle_length=tf.data.AUTOTUNE,
                               block_length=1)
    val_ds = val_ds.map(lambda record, filename: parse_tfrecord_function(record, filename, table, num_classes),
                        num_parallel_calls=tf.data.AUTOTUNE)
    val_ds = val_ds.batch(batch_size, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

    return train_ds, val_ds

def train():
    wandb.init()
    config = wandb.config

    # Set random seeds for reproducibility
    seed = 44
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # Define data paths for different sampling rates
    data_paths = {
        "timeStack1281281_1of1": "/content/timeStack_1281281_1of1_tf",
        "timeStack1281281_1of3": "/content/timeStack_1281281_1of3_tf",
        "timeStack1281281_1of6": "/content/timeStack_1281281_1of6_tf",
        "timeStack1281281_1of12": "/content/timeStack_1281281_1of12_tf",
        "timeStack1281281_1of24": "/content/timeStack_1281281_1of24_tf",
        "timeStack1281281_1of48": "/content/timeStack_1281281_1of48_tf",
        "timeStack1281281_1of96": "/content/timeStack_1281281_1of96_tf"
    }
    
    # Get configuration parameters
    data_name = config.get("data_name", "timeStack1281281")
    learning_rate = config.get("learning_rate", 0.0003)
    epochs = config.get("epochs", 90)
    batch_size = config.get("batch_size", 16)
    patience = config.get("patience", 100)
    min_delta = config.get("min_delta", 0.01)
    optimizer_name = config.get("optimizer","adam")
    l2_reg = config.get("dense_l2", 1e-4)
    dropout_rate = config.get("dense_dropout", 0.3)

    # Create datasets
    train_dataset, val_dataset = create_dataset_tf2(data_paths[data_name], batch_size, seed)

    # Build model architecture
    input_tensor = Input(shape=(224, 224, 3))
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model.trainable = True
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_reg))(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(10, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=outputs)

    # Configure optimizer
    if optimizer_name == "adam":
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "sgd_0.9":
      optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    elif optimizer_name == "sgd_0.95":
      optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.95)
    else:
      raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Compile model
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Learning rate scheduler
    def scheduler(epoch, lr):
        if epoch < 20:
            return learning_rate * (epoch + 1) / 20  # Linear warmup
        elif epoch < 80:
            T = 60
            cos_inner = np.pi * (epoch - 20) / T
            return learning_rate * (np.cos(cos_inner) + 1) / 2  # Cosine decay
        else:
            return learning_rate * 0.01  # Final small learning rate

    # Set up callbacks
    lr_callback = LearningRateScheduler(scheduler)
    checkpoint_path = f"/content/models/best{data_name}_{learning_rate}_{batch_size}_{dropout_rate}_{l2_reg}_{optimizer_name}_mbNetV2.h5"
    early_stop = EarlyStopping(monitor='val_accuracy', min_delta=min_delta, patience=patience, restore_best_weights=True)
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1)
    wandb_callback = CustomWandbCallback(save_model=False, log_graph=False)

    # Train model
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset,
        callbacks=[lr_callback, early_stop, checkpoint, wandb_callback]
    )

    # Evaluate model
    eval_results = model.evaluate(val_dataset)
    wandb.log({"Test Loss": eval_results[0], "Test Accuracy": eval_results[1]*100})

    # Generate confusion matrix
    valid_classes = sorted([cls for cls in os.listdir(data_paths[data_name])
                        if os.path.isdir(os.path.join(data_paths[data_name], cls)) and cls != '.ipynb_checkpoints'])
    y_preds = []
    y_trues = []
    for images, labels in val_dataset:
        preds = model.predict(images)
        y_preds.extend(np.argmax(preds, axis=1))
        y_trues.extend(np.argmax(labels.numpy(), axis=1))
    cm = metrics.confusion_matrix(y_trues, y_preds)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=valid_classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.savefig("confusion_matrix.png")
    wandb.log({"Confusion Matrix": wandb.Image("confusion_matrix.png")})

    wandb.finish()

In [ ]:
# Step 3: Define sweep configuration for hyperparameter optimization
sweep_config = {
    "program": "train.py",  # This doesn't affect the agent running our train() function
    "method": "grid",  # Options: "grid", "random", "bayes" etc.
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"  # We want to maximize validation accuracy
    },
    "parameters": {
        # Different sampling rates for event camera data
        "data_name": {
            "values": ["timeStack1281281_1of1", "timeStack1281281_1of3", "timeStack1281281_1of6",
                       "timeStack1281281_1of12", "timeStack1281281_1of24",
                       "timeStack1281281_1of48", "timeStack1281281_1of96"]
        },
        # Learning rate configuration
        "learning_rate": {
            "values": [3e-3]  # Fixed learning rate of 0.003
        },
        # Training epochs
        "epochs": {
            "value": 100  # Fixed number of epochs
        },
        # Batch size for training
        "batch_size": {
            "values": [16]  # Fixed batch size
        },
        # Early stopping parameters
        "patience": {
            "value": 100  # Number of epochs to wait before early stopping
        },
        "min_delta": {
            "value": 0.01  # Minimum change in monitored value to qualify as an improvement
        },
        # Regularization parameters
        "dense_dropout": {
            "values": [0.5]  # Dropout rate for dense layers
        },
        "dense_l2": {
            "values": [1e-3]  # L2 regularization strength
        },
        # Optimizer selection
        "optimizer": {
            "values": ["sgd_0.9"]  # SGD with momentum of 0.9
        }
    }
}

# Create sweep for hyperparameter optimization
sweep_id = wandb.sweep(sweep_config, project="event_MT_tf2mobileNetV2_sweep")
print("Created sweep with ID:", sweep_id)

In [ ]:
# Step 4：start sweep agent
wandb.agent(sweep_id, function=train)